In [1]:
import numpy as np
import cv2
import sys
import argparse
from scipy.spatial.transform import Rotation as R
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt 
import random, serial, time
import ipywidgets as widgets
from IPython.display import clear_output
 
# Connect to the device
import serial
import binascii
import struct


vib1 = False
vib2 = False

vib1_last = False
vib2_last = False


#####################

def send_angles(ang1, ang2):
    #print("sending")
    mo=4
    s.write(bytes(str(mo) + "," +str(ang1) + "," + str(ang2)+ "\n",'utf-8'))
    #print("sending_angles: "+ str(ang1) + "," + str(ang2)+ "\n")
    s.flushInput()

    
def read_angles():
    ang1 = 0
    ang2 = 0
    mo= 1
    s.write(bytes(str(mo) + "," +str(0) + "," + str(0)+ "\n",'utf-8'))
    a = s.readline().decode("utf-8") 
    try:
        ang1, ang2 = a.split(",")
        ang1 = int(ang1)
        ang2 = int(ang2)
    except:
        pass
    s.flushInput()
    return ang1, ang2
    
def send_to_home():
    ang1 = 90
    ang2= 90
    mo= 4
    s.write(bytes(str(mo) + "," + str(ang1) + "," + str(ang2)+ "\n",'utf-8'))
    print(str(ang1) + "," + str(ang2)+ "\n")
    s.flushInput()
    
def vib_1(vibr1):
    #print("sending")
    mo= 2
    s.write(bytes(str(mo) + "," + str(vibr1) + "," + str(vibr1)+ "\n",'utf-8'))
    print("vib_1: " + str(vibr1) + "," + str(vibr1)+ "\n")
    s.flushInput()

def vib_2(vibr2):
    #print("sending")
    mo= 3
    s.write(bytes(str(mo) + "," + str(vibr2) + "," + str(vibr2)+ "\n",'utf-8'))
    print("vib_2: " +str(vibr2) + "," + str(vibr2)+ "\n")
    s.flushInput()

def vib_on_off(v1,v2):
    #print("vibration88888888888888888888888888")
    if v1:
        vib_1(100)
    else:
        vib_1(0)
    if v2:
        vib_2(100)
    else:
        vib_2(0)

# send_angles(ang1,ang2,t)

# ang1 is the angle of the servo holding the arUco marker
# ang2 is the angle of the servo on the base
# t is the sleep time

def pose_esitmation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients,cal=False):
 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.aruco_dict = cv2.aruco.Dictionary_get(aruco_dict_type)
    parameters = cv2.aruco.DetectorParameters_create()


    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, cv2.aruco_dict,parameters=parameters,
        cameraMatrix=matrix_coefficients,
        distCoeff=distortion_coefficients)

        # If markers are detected
    found=False
    if len(corners) > 0:
        found=True
        for i in range(0, len(ids)):
            # Estimate pose of each marker and return the values rvec and tvec---(different from those of camera coefficients)
            if not cal:
                rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i],0.048 , matrix_coefficients,#0.048
                                                                           distortion_coefficients)
                                                                       
            else:                                                         
                rvec, tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(corners[i], 0.097, matrix_coefficients,
                                                                           distortion_coefficients)
            # Draw a square around the markers
            cv2.aruco.drawDetectedMarkers(frame, corners) 

            # Draw Axis
            cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.01)
            RR= R.from_rotvec(rvec[0])
            if cal:
                np.save("rvec_base", np.array(RR.as_matrix()))
                np.save("tvec_base", tvec.reshape(3,-1))
            else:
                np.save("rvec_tool", np.array(RR.as_matrix()))
                np.save("tvec_tool", tvec.reshape(3,-1))  
            RR= R.from_rotvec(rvec[0])
    if found:
        return frame,np.array(RR.as_matrix()),tvec.reshape(3,-1),found
    else:
        return frame,np.zeros(3),np.zeros(3), found
    
    
    
def Grid_search(seen):
    Gridwedth=12
    Gridhight=9
    send_angles(90,90)
    for i in range(Gridhight):
        if seen:
            break
        if i%2==0:
            start=5
            end=170
            k=1
        else:
            start=170
            end=5
            k=-1
        for j in range(Gridwedth):
            if seen:
                break
            send_angles(start+k*j*15,90-(i+1)*10)
            time.sleep(0.2)
            
            

def device_loc():
    global s, aru_x, aru_y, aru_z, interval,shown,vib1,vib2, vib1_last, vib2_last
    try:
        print("try")

        s = serial.Serial('COM5', baudrate=115200, timeout=0.1)
        print("done")
    except serial.SerialException:
        print("exp")

        s = serial.Serial('COM5', baudrate=115200, timeout=0.1)
        pass
        print('eeeror')
    
    k = np.array([[510.77, 0., 311.389],[0., 515.234, 243.123],[0., 0., 1.]])
    d = np.array([[0.085, -0.199, -0.00065, 0.00063, 0.0469]])
    aruco_dict_type = cv2.aruco.DICT_5X5_100
    video = cv2.VideoCapture(1)

    ang_x=np.empty(0)
    ang_y=np.empty(0)


    curent_angle_y = 90
    curent_angle_x = 90

    refrence_y=70
    refrence_x=110
    last_x=refrence_x
    send_angles(curent_angle_x,curent_angle_y)
    
    time.sleep(2.0)
    
    video.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    video.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    rvec_base=np.load("rvec_base.npy")
    tvec_base=np.load("tvec_base.npy")
    while True:
#         clear_output(wait=True)
        ret, frame = video.read()
        output,rvec_tool,tvec_tool,found = pose_esitmation(frame, aruco_dict_type, k, d,cal=False)
        
        if not ret:
            break
        
        if not found:
            shown=False
            
        if found:
            shown=True
            T_CB=np.array([[rvec_base[0,0,0],rvec_base[0,0,1],rvec_base[0,0,2],tvec_base[0,0]],[rvec_base[0,1,0],rvec_base[0,1,1],rvec_base[0,1,2],tvec_base[1,0]],  [rvec_base[0,2,0],rvec_base[0,2,1],rvec_base[0,2,2],tvec_base[2,0]],[0,0,0,1]])
            T_CT=np.array([[rvec_tool[0,0,0],rvec_tool[0,0,1],rvec_tool[0,0,2],tvec_tool[0,0]],[rvec_tool[0,1,0],rvec_tool[0,1,1],rvec_tool[0,1,2],tvec_tool[1,0]],[rvec_tool[0,2,0],rvec_tool[0,2,1],rvec_tool[0,2,2],tvec_tool[2,0]],[0,0,0,1]])
            T_transl=np.array([[1,0,0,0.1525],[0,1,0,0],[0,0,1,-0.0107],[0,0,0,1]])
            T_BC=np.linalg.inv(T_CB)
            T_final=T_transl @ T_BC @ T_CT
            ##### Find the values of the position of the ArUco marker in the robot's coordinate system. 
            aru_x = T_final[0,3]
            aru_y = T_final[1,3]
            aru_z = T_final[2,3]
            #tcp=R.from_matrix(rvec_tool)
            thetay=np.arccos(rvec_tool[0,1,2])*180/np.pi
            thetax=np.arccos(rvec_tool[0,0,2])*180/np.pi
            
            if(np.absolute(thetax-refrence_x)>5):
                if (thetax>refrence_x):
                    curent_angle_x=curent_angle_x+0.5

                    if curent_angle_x>175:
                        curent_angle_x=175;
                else:
                    curent_angle_x=curent_angle_x-0.5
                    if curent_angle_x<3:
                        curent_angle_x=3;
                send_angles(curent_angle_x,curent_angle_y)

            ang_x=np.append(ang_x,90-thetax)
            np.save("xang",ang_x)

            if(np.absolute(thetay-refrence_y)>5):
                if(thetay>refrence_y):
                    curent_angle_y=curent_angle_y+0.5
                    if curent_angle_y<3:
                        curent_angle_y=3
                else:
                    curent_angle_y=curent_angle_y-0.5
                    if curent_angle_y>175:
                        curent_angle_y=175
                send_angles(curent_angle_x,curent_angle_y)

            ang_y=np.append(ang_y,90-thetay)
            np.save("yang",ang_y)


        cv2.imshow('Estimated Pose', frame)

        key = cv2.waitKey(1) & 0xFF 
        if key == ord('q'):
            s.close()
            cv2.destroyAllWindows()
            break  

In [2]:
############### Experiment 2  ##################  OK  #avoidance controller 2 + trajectory points BT + tracker + selfdriving    ###### WITH haptic device

import time, serial
import sys
import urx
import numpy as np
import json, math
import numpy as np
from numpy.linalg import inv
import os
from scipy import stats
import  socket
import robotiq_gripper
from IPython.display import clear_output

from _thread import *
import threading

global df1, df, aru_x, aru_y, aru_z, interval,vib1,vib2,shown

vib1=False
vib2=False
shown=False
interval = True


gripper = robotiq_gripper.RobotiqGripper()
print("Connecting to gripper...")
gripper.connect("192.168.88.139", 63352)
grip_pos=100
gripper.move(grip_pos, 100, 100)


ef= open("exper2_Fikre_25_1.txt","w+")


try:
    rob = urx.Robot("192.168.88.139")
except:
    print('robot error')
    interval = False


### thread to read the hand position always
p = threading.Thread(target=device_loc, args=())
p.start()
time.sleep(5.0)


if interval:

    #Controller
    Px_c=np.empty(0)
    Py_c=np.empty(0)
    Pz_c=np.empty(0)
    Px_arm=np.empty(0)
    Py_arm=np.empty(0)
    Pz_arm=np.empty(0)
    P = np.array([0.2,0.2,0.2]).reshape(3,1)
    rad_error=np.empty(0)
    P_obs = np.array([0.2,0.2,0.2]).reshape(3,1)
    V = np.array([0.0,0.0,0.0]).reshape(3,1)
    Po = np.array([2.1,2.1,10.1]).reshape(3,1)
    e = np.zeros((3,1))
    ea = np.zeros((3,1))
    lamda = 0.01
    k2 = (np.diag([1.2,2.3,3.3]))
    k1 = ( np.diag([1.1,2.3,1.3]))
    k3 = ( np.diag([1.1,2.3,1.3]))
    time_r=np.empty(0)
    with open('poses_aruco2.json', 'r') as fa:
        data = json.load(fa)
        n_points = data['general']['n']
        n_counter = 1
        x_refr = data['position'+str(n_counter)]['xref']
        y_refr = data['position'+str(n_counter)]['yref']
        z_refr = data['position'+str(n_counter)]['zref']
        rx_refr = data['position'+str(n_counter)]['Rxref']
        ry_refr = data['position'+str(n_counter)]['Ryref']
        rz_refr = data['position'+str(n_counter)]['Rzref']
        
        
    rob.movel((x_refr, y_refr, z_refr, rx_refr,ry_refr,rz_refr ), 0.1,0.1)
    
    to = time.time()
        
    while True:
        tic = time.time()
        
        clear_output(wait=True)
        
        try:
            with open('data2.json', 'r') as fa:
                data = json.load(fa)
                k1[0,0] = data['controller']['k11']
                k1[1,1] = data['controller']['k12']
                k1[2,2] = data['controller']['k13']
                k2[0,0] = data['controller']['k21']
                k2[1,1] = data['controller']['k22']
                k2[2,2] = data['controller']['k23']
                k3[0,0] = data['controller']['k31']
                k3[1,1] = data['controller']['k32']
                k3[2,2] = data['controller']['k33']
                vmax = data['controller']['vmax']
                vmax2 = data['poss_obs']['vmax']
                T = data['poss']['T']
                dmin = data['poss_obs']['dmin']
                dmin2 = data['poss_obs']['dmin2']
                kd1 = data['poss_obs']['kd1']
                kd2 = data['poss_obs']['kd2']
                kd3 = data['poss_obs']['kd3']
                tau = data['poss_obs']['tau']
                kangle = data['poss_obs']['kangle']
        except:
            print('file error')


        try:
            
            ################
            ##########  Get position from Aruco markers
            
            xo_refr =aru_x
            yo_refr = aru_y 
            zo_refr = aru_z
            Px_c=np.append(Px_c,aru_x)
            Py_c=np.append(Py_c,aru_y)
            Pz_c=np.append(Pz_c,aru_z)
            np.save("Px_c", Px_c)
            np.save("Py_c", Py_c)
            np.save("Pz_c", Pz_c)
            
        except:
            xo_refr = 0
            yo_refr = 0
            zo_refr = 0 

        
       
        pose = rob.getl()
        xn = float(pose[0])
        yn = float(pose[1])
        zn = float(pose[2])
        
        Px_arm=np.append(Px_arm,xn)
        Py_arm=np.append(Py_arm,yn)
        Pz_arm=np.append(Pz_arm,zn)
        np.save("Px_arm", Px_arm)
        np.save("Py_arm", Py_arm)
        np.save("Pz_arm", Pz_arm)  
        
        
        P = np.zeros((3,1));
        P[0,0] = float(xn)
        P[1,0] = float(yn)
        P[2,0] = float(zn)
        print('Actual Point',P)
        
        ex = xo_refr-xn
        ey = yo_refr-yn
        ez = zo_refr-zn
        
        print("ex = ",ex,"ey = ",ey,"ez = ",ez)
 
        
        
        
        
        if np.linalg.norm(e) < 0.01 :
            n_counter = n_counter + 1
            if n_counter > n_points :
                n_counter = 1
                
            with open('poses_aruco2.json', 'r') as fa:
                
                    data = json.load(fa)
                    n_points = data['general']['n']
                    x_refr = data['position'+str(n_counter)]['xref']
                    y_refr = data['position'+str(n_counter)]['yref']
                    z_refr = data['position'+str(n_counter)]['zref']
                    action = data['position'+str(n_counter)]['action']
                
                    if action == "open":
                        gripper.move(100, 300, 100)
                        time.sleep(3)
                    elif action == "close":
                        gripper.move(215, 300, 100)
                        time.sleep(3)
                    elif action == "openliq":
                        gripper.move(195 , 300, 100)
                        time.sleep(3)
                    elif action=="end":
                        break
            
            
        Pa = np.zeros((3,1));    
        Pa[0,0] = float(x_refr)
        Pa[1,0] = float(y_refr)
        Pa[2,0] = float(z_refr)

        
        P_obs = np.zeros((3,1));   
        P_obs[0,0] = float(xo_refr)
        P_obs[1,0] = float(yo_refr)
        P_obs[2,0] = float(zo_refr)

        e = Pa - P      
        
        hv = np.matmul(k2,np.tanh(np.matmul(k1,e) + np.matmul(k3,e-ea)))
        ea = e
        
        r_ao = P_obs - P
        n_r_ao = np.linalg.norm(r_ao)  #### distance from robot to the obstacle
        print("n_r_ao: ", n_r_ao)
        rad_error=np.append(rad_error,n_r_ao)
        np.save("rad_error",rad_error)
        if n_r_ao> dmin :
            vib1=False
            vib2=False
        if n_r_ao < dmin and n_r_ao > dmin2 - 0.05:
            
            #First collision condition
            print('collision condition 1')
            teta = np.arccos(np.matmul(r_ao.transpose(),e)/ (np.linalg.norm(r_ao)*np.linalg.norm(e)) )
            alfa = np.arccos(np.matmul(r_ao.transpose(),-P)/ (np.linalg.norm(r_ao)*np.linalg.norm(P)) ) 
            beta = np.arccos(np.matmul(e.transpose(),-P)/ (np.linalg.norm(P)*np.linalg.norm(e)) )
            vib1=True
            vib2=False
            
            if  teta < np.pi*kangle/180:
                
                #second collision condition
                nr = P / np.linalg.norm(P)
                print('collision condition 2')
                nrg = e / np.linalg.norm(e)
                nv = V / np.linalg.norm(V)
                nro = r_ao / np.linalg.norm(r_ao)
                
                
                # SPEED
                
                a = np.cross(nro.flatten(),nv.flatten())
                b = np.cross(a.flatten(),nro.flatten())
                b = b /np.linalg.norm(b)
                P_aux = np.zeros((3,1)) 
                P_aux[0,0] = -float(xn)
                P_aux[1,0] = -float(yn) 
                ang_aux = np.arccos(np.matmul(b,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b)) )
                print('aux',ang_aux)
                
                if ang_aux > np.pi/2 :
                    b = b
                else :
                    b = -b
  
                
                # AXIS Z
                
                a1 = np.cross(nro.flatten(),P_aux.flatten())
                b1 = np.cross(a1.flatten(),nro.flatten())
                b1 = b1 /np.linalg.norm(b1)
                ang_aux = np.arccos(np.matmul(b1,nr)/ (np.linalg.norm(nr)*np.linalg.norm(b1)) )
                print('aux',ang_aux)
                if ang_aux > np.pi/2 :
                    b1 = b1
                else :
                    b1 = -b1
                    
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Fd = np.matmul(V.reshape([1,3]),ur.reshape([3,1]))
                Ft = kd1*b.reshape([3,1])*Fk + kd2*b1.reshape([3,1])*Fk - kd3*Fk*ur.reshape([3,1]) 
                
                vx1 = np.exp(-dmin*tau)*Ft[0,0] + (1-np.exp(-dmin*tau))*hv[0,0]
                vy1 = np.exp(-dmin*tau)*Ft[1,0] + (1-np.exp(-dmin*tau))*hv[1,0]
                vz1 = np.exp(-dmin*tau)*Ft[2,0] + (1-np.exp(-dmin*tau))*hv[2,0]
            else:                  
                ur = r_ao/np.linalg.norm(r_ao)
                Fk = (1/(np.power(n_r_ao,3)))
                Ft = - kd3*Fk*ur.reshape([3,1]) 
                vx1 = 0.8*np.exp(-dmin*tau)*hv[0,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[0,0]
                vy1 = 0.8*np.exp(-dmin*tau)*hv[1,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[1,0]
                vz1 = 0.8*np.exp(-dmin*tau)*hv[2,0] + 1.2*(1-np.exp(-dmin*tau))*Ft[2,0]
            
        elif n_r_ao < dmin2 or not shown  :#or not shown
            sta=time.time()
            print("danger")
            vx1 = 0
            vy1 = 0
            vz1 = 0
            rob.set_freedrive(1)
            vib2=True
            vib1=True
            while n_r_ao < dmin2:#+0.1:
                try:

              ############ read position of the obstacle

                    en=time.time()
                    xo_refr = aru_x
                    yo_refr = aru_y 
                    zo_refr = aru_z
                    Px_c=np.append(Px_c,aru_x)
                    Py_c=np.append(Py_c,aru_y)
                    Pz_c=np.append(Pz_c,aru_z)
                    
                    np.save("Px_c", Px_c)
                    np.save("Py_c", Py_c)
                    np.save("Pz_c", Pz_c)
                    
                except:
                    xo_refr = 0
                    yo_refr = 0
                    zo_refr = 0



                pose = rob.getl()
                xn = float(pose[0])
                yn = float(pose[1])
                zn = float(pose[2])
                
                Px_arm=np.append(Px_arm,xn)
                Py_arm=np.append(Py_arm,yn)
                Pz_arm=np.append(Pz_arm,zn)
                
            
                np.save("Px_arm", Px_arm)
                np.save("Py_arm", Py_arm)
                np.save("Pz_arm", Pz_arm) 
                
                
                P = np.zeros((3,1));
                P[0,0] = float(xn)
                P[1,0] = float(yn)
                P[2,0] = float(zn)



                P_obs = np.zeros((3,1));   
                P_obs[0,0] = float(xo_refr)
                P_obs[1,0] = float(yo_refr)
                P_obs[2,0] = float(zo_refr)

                r_ao = P_obs - P
                n_r_ao = np.linalg.norm(r_ao)
                rad_error=np.append(rad_error,n_r_ao)
                np.save("rad_error",rad_error)
                rob.set_freedrive(1)
                
                
            rob.set_freedrive(0)
        
        
        else :
            vx1 = hv[0,0]
            vy1 = hv[1,0]
            vz1 = hv[2,0]
            vib1=False
            vib2=False
        
        
        
        VMAX = np.sqrt(vx1*vx1+vy1*vy1+vz1*vz1) 
        
        if (VMAX > vmax):
            vx1 = vx1*vmax/VMAX
            vy1 = vy1*vmax/VMAX
            vz1 = vz1*vmax/VMAX

        

        V[0,0] = vx1
        V[1,0] = vy1
        V[2,0] = vz1

        
        try:
            print("vx1: ", vx1)
            print("vy1: ", vy1)
            print("vy1: ", vz1)
            
            print("arx: ", xo_refr)
            print("ary: ", yo_refr)
            print("arz: ", zo_refr)
            rob.speedl((vx1*0.2 , vy1*0.2, vz1*0.2 , 0, 0, 0), acc=0.4, min_time=1)
        except:
            print('sent')

        time.sleep(0.1)

        toc = time.time()
        
        time_r=np.append(time_r,toc-tic)
        np.save("time_r",time_r)
        print("tic",tic)
        print("toc",toc)
        
    fin=time.time()
    print("to",to)
    print("fin",fin)
else:
    print('finishing')
    try:
        rob.stopl()
    
    except:
        print('No')
        
    
        


Actual Point [[ 0.82396675]
 [-0.43946414]
 [ 0.29170098]]
ex =  -0.17430099880469252 ey =  0.5210107384839123 ez =  0.022537510129142624
to 1683052730.907662
fin 1683053007.5483866
